<a href="https://colab.research.google.com/github/marco-siino/Enhanced-Home-Appliance-Scheduling-with-GPT/blob/main/Codestral/Codestral_EnergyCostOptimization_Evaluator_MSiino.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import the libraries required.

In [1]:
import requests
import zipfile
import io
import calendar
import re
from datetime import datetime

# Get the log files from GitHub and extract the folders.

In [2]:
def get_log_files_zipped (zip_url, folder_name):
  # Send a GET request to the URL
  response = requests.get(zip_url)

  # Check if the request was successful (status code 200)
  if response.status_code == 200:
      # Read the content of the response
      zip_content = io.BytesIO(response.content)

      # Extract the zip file
      with zipfile.ZipFile(zip_content, 'r') as zip_ref:
          zip_ref.extractall('/content/'+folder_name)  # Extract to the '/content/' directory in Colab
      print("Zip file extracted successfully.")
  else:
      print("Failed to download zip file.")

In [3]:
# GitHub repository URL
jul2014_url = 'https://github.com/marco-siino/Enhanced-Home-Appliance-Scheduling-with-GPT/raw/refs/heads/main/Codestral/multiple_appliances_jul14.zip'
nov2014_url = 'https://github.com/marco-siino/Enhanced-Home-Appliance-Scheduling-with-GPT/raw/refs/heads/main/Codestral/multiple_appliances_nov14.zip'
feb2015_url = 'https://github.com/marco-siino/Enhanced-Home-Appliance-Scheduling-with-GPT/raw/refs/heads/main/Codestral/multiple_appliances_feb15.zip'
apr2015_url = 'https://github.com/marco-siino/Enhanced-Home-Appliance-Scheduling-with-GPT/raw/refs/heads/main/Codestral/multiple_appliances_apr15.zip'

get_log_files_zipped(jul2014_url,"jul2014")
get_log_files_zipped(nov2014_url,"nov2014")
get_log_files_zipped(feb2015_url,"feb2015")
get_log_files_zipped(apr2015_url,"apr2015")



Zip file extracted successfully.
Zip file extracted successfully.
Zip file extracted successfully.
Zip file extracted successfully.


# Function to get the content of a log file.

In [4]:
def get_log_file_content(day,month,year):
  month_number = int(month)
  month_name = calendar.month_name[month_number]
  # Get the first three characters of the month name
  short_month_name = month_name[:3].lower()  # Convert to lowercase for consistency
  file_path = short_month_name+str(year)+"/multiple_appliances/multiple_appliances_"+str(year)+"-"+str(month)+"-"+str(day)+".txt"
  file_content=""
  print(file_path)
  try:
    # Open the file in read mode
    with open(file_path, 'r') as file:
        # Read the contents of the file into a string
        file_content = file.read()
        print("File content stored in string successfully.")
  except FileNotFoundError:
    print("File not found. Please provide the correct file path.")
  except Exception as e:
    print("An error occurred:", e)
  return file_content

# Given a daily log file, function to count and return the number of actual intervals and the number of Mistral-proposed intervals.

In [9]:
def calculate_missing_intervals(day,month,year):
  file_content = get_log_file_content(day,month,year)
  split_string = file_content.split("Codestral operating times:", 1)

  actual_intervals = split_string[0]
  mistral_intervals = split_string[1]

  nr_actual_intervals = actual_intervals.count("From")
  nr_mistral_intervals = mistral_intervals.count("From")

  return nr_actual_intervals,nr_mistral_intervals

# Given a daily log file, function to parse the Mistral proposal and to evaluate if a given interval is acceptable or not.
The function returns the total number of proposed intervals and the number of acceptable intervals.

In [13]:
def calculate_acceptable_intervals(day,month,year):
  file_content = get_log_file_content(day,month,year)
  split_string = file_content.split("Codestral operating times:", 1)

  mistral_intervals = split_string[1]

  lines = mistral_intervals.split('\n')

  nr_acceptable_intervals=0
  for line in lines:
        # Define the regular expression pattern to match words in the format xx:xx:xx
        pattern = r'\b\d{2}:\d{2}:\d{2}\b'

        # Find all occurrences of the pattern in the input string
        matches = re.findall(pattern, line)

        # Extract the first and second occurrences
        from_time = matches[0] if matches else None
        to_time = matches[1] if len(matches) > 1 else None

        try:
          if (from_time != None)  and (to_time!=None):
            if from_time < to_time:
              nr_acceptable_intervals+=1
        except:
          continue

  return nr_acceptable_intervals

# Function to compute the evaluation metrics given a month.

In [14]:
def compute_evaluation_metrics(month,year):
  nr_total_actual_intervals = 0
  nr_total_mistral_proposed_intervals = 0
  nr_total_acceptable_intervals = 0
  last_day = 31

  # February 2015
  if month=="02" and year=="2015":
    last_day = 28
  if month=="04":
    last_day = 30
  if month=="07":
    last_day = 31
  if month=="11":
    last_day = 30

  for day in range(1,last_day+1):
    two_digits_day = "{:02d}".format(day)
    current_actual_intervals,current_mistral_proposed_intervals = calculate_missing_intervals(two_digits_day,month,year)
    nr_total_actual_intervals += current_actual_intervals
    nr_total_mistral_proposed_intervals += current_mistral_proposed_intervals
    nr_total_acceptable_intervals+=calculate_acceptable_intervals(two_digits_day,month,year)
    try:
      print("Current missing ratio: "+str((1-(current_mistral_proposed_intervals/current_actual_intervals))))
    except:
      print("No actual intervals are present.")

    try:
      print("Current acceptability: "+str((calculate_acceptable_intervals(two_digits_day,month,year)/current_mistral_proposed_intervals)))
    except:
      print("No mistral proposals are present.")
    print("\n\n")

  missing_intervals_ratio = 1-(nr_total_mistral_proposed_intervals/nr_total_actual_intervals)
  accuracy = nr_total_acceptable_intervals/nr_total_mistral_proposed_intervals

  print("The acceptability for "+month+ "/"+year+" is: "+str(accuracy))
  print("The missing intervals ratio for "+month+ "/"+year+" is: "+str(missing_intervals_ratio))

In [15]:
compute_evaluation_metrics("02","2015")

feb2015/multiple_appliances/multiple_appliances_2015-02-01.txt
File content stored in string successfully.
feb2015/multiple_appliances/multiple_appliances_2015-02-01.txt
File content stored in string successfully.
Current missing ratio: 0.0
feb2015/multiple_appliances/multiple_appliances_2015-02-01.txt
File content stored in string successfully.
Current acceptability: 0.9090909090909091



feb2015/multiple_appliances/multiple_appliances_2015-02-02.txt
File content stored in string successfully.
feb2015/multiple_appliances/multiple_appliances_2015-02-02.txt
File content stored in string successfully.
Current missing ratio: 0.0
feb2015/multiple_appliances/multiple_appliances_2015-02-02.txt
File content stored in string successfully.
Current acceptability: 1.0



feb2015/multiple_appliances/multiple_appliances_2015-02-03.txt
File content stored in string successfully.
feb2015/multiple_appliances/multiple_appliances_2015-02-03.txt
File content stored in string successfully.
Current missing

In [18]:
compute_evaluation_metrics("07","2014")

jul2014/multiple_appliances/multiple_appliances_2014-07-01.txt
File content stored in string successfully.
jul2014/multiple_appliances/multiple_appliances_2014-07-01.txt
File content stored in string successfully.
Current missing ratio: 0.0
jul2014/multiple_appliances/multiple_appliances_2014-07-01.txt
File content stored in string successfully.
Current acceptability: 1.0



jul2014/multiple_appliances/multiple_appliances_2014-07-02.txt
File content stored in string successfully.
jul2014/multiple_appliances/multiple_appliances_2014-07-02.txt
File content stored in string successfully.
Current missing ratio: 0.0
jul2014/multiple_appliances/multiple_appliances_2014-07-02.txt
File content stored in string successfully.
Current acceptability: 1.0



jul2014/multiple_appliances/multiple_appliances_2014-07-03.txt
File content stored in string successfully.
jul2014/multiple_appliances/multiple_appliances_2014-07-03.txt
File content stored in string successfully.
Current missing ratio: 0.0
jul

In [17]:
compute_evaluation_metrics("11","2014")

nov2014/multiple_appliances/multiple_appliances_2014-11-01.txt
File content stored in string successfully.
nov2014/multiple_appliances/multiple_appliances_2014-11-01.txt
File content stored in string successfully.
Current missing ratio: 0.0
nov2014/multiple_appliances/multiple_appliances_2014-11-01.txt
File content stored in string successfully.
Current acceptability: 1.0



nov2014/multiple_appliances/multiple_appliances_2014-11-02.txt
File content stored in string successfully.
nov2014/multiple_appliances/multiple_appliances_2014-11-02.txt
File content stored in string successfully.
Current missing ratio: 0.0
nov2014/multiple_appliances/multiple_appliances_2014-11-02.txt
File content stored in string successfully.
Current acceptability: 1.0



nov2014/multiple_appliances/multiple_appliances_2014-11-03.txt
File content stored in string successfully.
nov2014/multiple_appliances/multiple_appliances_2014-11-03.txt
File content stored in string successfully.
Current missing ratio: 0.0
nov

In [16]:
compute_evaluation_metrics("04","2015")

apr2015/multiple_appliances/multiple_appliances_2015-04-01.txt
File content stored in string successfully.
apr2015/multiple_appliances/multiple_appliances_2015-04-01.txt
File content stored in string successfully.
Current missing ratio: 0.0
apr2015/multiple_appliances/multiple_appliances_2015-04-01.txt
File content stored in string successfully.
Current acceptability: 1.0



apr2015/multiple_appliances/multiple_appliances_2015-04-02.txt
File content stored in string successfully.
apr2015/multiple_appliances/multiple_appliances_2015-04-02.txt
File content stored in string successfully.
Current missing ratio: 0.0
apr2015/multiple_appliances/multiple_appliances_2015-04-02.txt
File content stored in string successfully.
Current acceptability: 1.0



apr2015/multiple_appliances/multiple_appliances_2015-04-03.txt
File content stored in string successfully.
apr2015/multiple_appliances/multiple_appliances_2015-04-03.txt
File content stored in string successfully.
No actual intervals are presen